Import module

In [ ]:
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 560 kB/s 


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer
import tensorflow.keras.utils as ku 
import random
import ftfy
import nltk
nltk.download("stopwords")
nltk.download("punkt")
import re
from nltk.corpus import stopwords
from nltk import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


import data dari google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Fungsi Penting**


In [ ]:
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            # remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(
                re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

            # fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)

            # expand contraction
            tweet = expandContractions(tweet)

            # remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            # stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet)
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            # stemming words
            tweet = PorterStemmer().stem(tweet)

            cleaned_tweets.append(tweet)

    return cleaned_tweets

#Membuat dataset

>**Depression on Twitter**

We use three dataset pegged into one, for noting bad condition we use:

*   depression dataset:
https://github.com/eddieir/Depression_detection_using_Twitter_post
*   suicidal dataset
https://github.com/laxmimerit/twitter-suicidal-intention-dataset

and for normal condition we use this dataset:
*   Sentiment Analysis Dataset 2.csv from https://www.kaggle.com/datasets/ywang311/twitter-sentiment

In [ ]:
depresi  = pd.read_csv('/content/drive/MyDrive/M_care/depressive_tweets_processed.csv', sep = '|', header = None, usecols = range(1,9), nrows = 3842)
suicidal = pd.read_csv('/content/drive/MyDrive/M_care/twitter-suicidal_data.csv')
non_bad      = pd.read_csv('/content/drive/MyDrive/M_care/Sentiment Analysis Dataset 2.csv', encoding = "ISO-8859-1", usecols = range(0,4), nrows = 12588)

In [ ]:
dp  = [x for x in depresi[5]]
sc  = [x for x in suicidal[suicidal.intention == 1].tweet]
nd  = [x for x in non_bad['SentimentText']]
X_dp = clean_tweets(dp)
X_sc = clean_tweets(sc)
X_nd = clean_tweets(nd)

In [ ]:
(len(X_dp[:-4])+len(X_sc[:-4]))

6294

In [ ]:
len(X_nd)

12497

In [ ]:
well  = X_nd[:6294].copy()

In [ ]:
nwell = X_dp[:-4].copy()+X_sc[:-4].copy()

In [ ]:
len(nwell)

6294

Deskripsi

In [ ]:
NDATA=int(6294/3)
data_1_1=nwell[       :NDATA*1]
data_1_2=nwell[NDATA*1:NDATA*2]
data_1_3=nwell[NDATA*2:NDATA*3]
data_1_4=well[NDATA*0:NDATA*1]
data_1_5=well[NDATA*1:NDATA*2]
data_1_6=well[NDATA*2:NDATA*3]

In [ ]:
tweet_1 = data_1_1+data_1_5
tweet_2 = data_1_3+data_1_4
tweet_3 = data_1_2+data_1_6

In [ ]:
len(tweet_3)/2

2098.0

In [ ]:
target=[0 for i in range(2098)]+[1 for i in range(2098)]

In [ ]:
data = {'tweet1': tweet_1, 'tweet2': tweet_2,'tweet3':tweet_3,'target':target}

In [ ]:
df = pd.DataFrame(data=data)

In [ ]:
df.to_csv('tiga_tweet.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2304 entries, 0 to 2303
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet1  2304 non-null   object
 1   tweet2  2304 non-null   object
 2   tweet3  2304 non-null   object
 3   target  2304 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 72.1+ KB
